In [1]:
import pandas as pd
import numpy as np
from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

C:\Conda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Load data
fire_df = pd.read_csv('Fire_data.csv')

#######CLEAN DATA##########

# only get rows in Butte, CA
fire_df = fire_df[fire_df['state']=='CA']
fire_df = fire_df[(fire_df['latitude']>=39.0) & (fire_df['latitude']<=41.0)]

# Remove uneccesary rows
fire_df = fire_df[['latitude', 'longitude', 'state', 'disc_clean_date', 'Temp_pre_15', 'Temp_pre_7', 'Wind_pre_15', 'Wind_pre_7','Hum_pre_15', 'Hum_pre_7','Prec_pre_15', 'Prec_pre_7']]
fire_df = fire_df.drop(fire_df[fire_df['Prec_pre_15']==-1].index)
fire_df = fire_df.drop(fire_df[fire_df['Prec_pre_7']==-1].index)

# Target row (fire): 1=Fire, 0=no fire
fire_df['fire']=1
wf_df = fire_df[['latitude', 'longitude', 'disc_clean_date']]
wf_df.to_csv('wildfire_data.csv')

In [3]:
ncaa = weather_df = pd.read_csv('ncaa_data.csv')
ncaa['fire']=0
ncaa['temprature'] = ncaa['LST'] - 273.15

In [4]:
weather_df = pd.read_csv('Weather.csv')
weather_df['fire']=0
weather_df['Temp_pre_7'] = ncaa['temprature']

weather_df = weather_df.dropna()

df = pd.concat([weather_df, fire_df], axis=0)
df = df.sample(frac = 1)
mat=df.corr()
# print(mat['fire'])

In [5]:
X = df[['Temp_pre_7', 'Wind_pre_7', 'Hum_pre_7']]
Y = df[['fire']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [6]:
base_classifier = DecisionTreeClassifier(max_depth=1)  # Weak learner for classification
n_estimators = 50  # Number of estimators
ada_classifier = AdaBoostClassifier(base_classifier, n_estimators=n_estimators)
ada_classifier.fit(X_train, y_train)
y_pred = ada_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.95      0.97        75
           1       0.97      1.00      0.98       129

    accuracy                           0.98       204
   macro avg       0.98      0.97      0.98       204
weighted avg       0.98      0.98      0.98       204



C:\Conda\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
